In [10]:
import sqlite3
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()

results = [row[0] for row in c.execute('SELECT verb FROM appdb ORDER BY freq DESC LIMIT 10')]
results

['andare',
 'aprire',
 'avere',
 'bere',
 'cadere',
 'conoscere',
 'dire',
 'dovere',
 'essere',
 'fare']

In [44]:
# A1 deck
QUERY = '''
  select a.*, c.ROWID, c.pos, c.verb, c.form, c.conjugation, f.level from cards c
  INNER JOIN appdb a ON c.verb = a.verb 
  INNER JOIN forms f ON f.form = c.form
  WHERE c.form = 'INDICATIVO_PASSATO_PROSSIMO' AND a.irregular
  ORDER BY RANDOM()
  '''

conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
results = [row for row in c.execute(QUERY)]

In [45]:
results[0:5]

[('fare',
  99999999,
  1,
  0,
  1048,
  7,
  'fare',
  'INDICATIVO_PASSATO_PROSSIMO',
  'loro(f) hanno fatto',
  'A1'),
 ('conoscere',
  99999999,
  1,
  0,
  11795,
  2,
  'conoscere',
  'INDICATIVO_PASSATO_PROSSIMO',
  'lui ha conosciuto',
  'A1'),
 ('potere',
  99999999,
  1,
  0,
  12358,
  5,
  'potere',
  'INDICATIVO_PASSATO_PROSSIMO',
  'voi avete potuto / voi siete potuti',
  'A1'),
 ('volere',
  99999999,
  1,
  0,
  1938,
  1,
  'volere',
  'INDICATIVO_PASSATO_PROSSIMO',
  'tu hai voluto',
  'A1'),
 ('dire',
  99999999,
  1,
  0,
  707,
  2,
  'dire',
  'INDICATIVO_PASSATO_PROSSIMO',
  'lui ha detto',
  'A1')]

In [3]:
import urllib
url = 'https://conjugator.reverso.net/index-italian-1-250.html'

def request_contents(url):
    try:
        contents = urllib.request.urlopen(url)
    except urllib.error.HTTPError:
        return (None)
    return (contents.read().decode(encoding="utf-8", errors="strict"))

In [5]:
html = request_contents(url)

HTTPException: got more than 100 headers

In [14]:
from cmd import Cmd

def show_card():
    print('''
    Presente: io canto
    
    futuro: io canterò
    ''')

class MyPrompt(Cmd):
    def __init__(self):
        super().__init__()
        self.counter = 12

    def do_increment(self, input):
        self.counter += 1
        
    def do_0(self, input):
        self.counter = 0
    
    def do_card(self, input):
        show_card()
    
    def do_exit(self, inp):
        print("Bye", self.counter)
        return True
 
    def do_add(self, inp):
        print("Adding '{}'".format(inp))
 
MyPrompt().cmdloop()

(Cmd) 0
(Cmd) exit
Bye 0


In [ ]:
import csv
from collections import defaultdict
import random
import os

url1 = '/Users/davidp/work/verbe-italiani/data/csvs/top236.csv'

forms = [
    'Condizionale Passato',
    'Condizionale Presente',
    'Congiuntivo Imperfetto',
    'Congiuntivo Perfetto',
    'Congiuntivo Presente',
    'Congiuntivo Trapassato',
    'Futuro',
    'Futuro Anteriore',
    'Imperfetto',
    'Passato Remoto',
    'Perfetto',
#    'Presente',
    'Trapassato Prossimo'
]

In [ ]:
random.choice(range(8))

In [ ]:
def write_full_csv(url, cards):
    with open(url, encoding='utf-8', mode='w') as f:
        w = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for card in cards:
            w.writerow(card)

In [ ]:
def read_full_csv(url):
    db = defaultdict(dict)
    with open(url, encoding='utf-8', mode='r') as f:
        r = csv.reader(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for i, row in enumerate(r):
            if i == 0:
                continue
            _, pos, root, form, front, back = row
            if not form in db[root].keys():
                db[root][form] = ['' for i in range(8)]            
            db[root][form][int(pos)] = back
    return(db)

In [ ]:
x = 'Abc'
'-'.join(x.lower().split(' '))

In [ ]:
def one_form(db, form):
    keys = list(db.keys())
    random.shuffle(keys)
    
    cards = []
    imperfetto = []
    for root in db.keys():
        presente = db[root]['Presente']
        i = random.choice(range(8))
        front = '%s --> %s' %(presente[i], form)
        back = db[root][form][i]
        cards += [[front, back]]
    return(cards)
    
def main(url):
    db = read_full_csv(url1)
    for form in forms:
        cards = one_form(db, form)
        base, ext = os.path.splitext(url)
        suffix = '-'.join(form.lower().split(' '))
        output = '%s-%s%s' %(base, suffix, ext)
        write_full_csv(output, cards)

main(url1)